[简书:利用Python进行数据分析](https://www.jianshu.com/p/04d180d90a3f)

In [1]:
import pandas as pd
import numpy as np

### 数据聚合与分组运算


In [2]:
df = pd.DataFrame({
    "key1" : ['a', 'a', 'b', 'b', 'a'],
    "key2" : ['one', 'two', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4],
    "value2" : [2, 6, 8, 10, 12]
})
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


#### GroupBy机制

- 分组键的形式，可以有多种形式且类型不必相同：
    - 列表或数组，其长度与待分组的轴一样。
    - 表示DataFrame某个列名的值。
    - 字典或Series，给出待分组轴上的值与分组名之间的对应关系。
    - 函数，用于处理轴索引或索引中的各个标签。


- 注意事项
    - 任何分组关键词中的缺失值，都会被从结果中除去。

- **按照key1进行数据分组并计算value1列的和**

In [4]:
# GroupBy对象，无数据计算，只是包含一些有关分组键的中间数据
df_grouped = df["value1"].groupby(df["key1"])
df_grouped

In [5]:
df_grouped.sum()

key1
a    5
b    5
Name: value1, dtype: int64

- **对两个关键字进行层次化索引**

In [5]:
df_multiGrouped = df["value1"].groupby([df["key1"], df["key2"]])
df_multiGrouped.sum()

key1  key2
a     one     4
      two     1
b     one     2
      two     3
Name: value1, dtype: int64

In [6]:
df_multiGrouped.sum().unstack()

key2,one,two
key1,,
a,4,1
b,2,3


- **将列名(可以是字符串、数字或Python对象)用作分组键**

In [7]:
df_multiGrouped2 = df.groupby(["key1", "key2"])
df_multiGrouped2.sum().unstack()

value1     value2    
key2    one two    one two
key1                      
a         4   1     14   6
b         2   3      8  10

In [8]:
df_multiGrouped2.size().unstack()

key2,one,two
key1,,
a,2,1
b,1,1



##### 对分组进行迭代


In [9]:
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


- **单键情况下，分组迭代**

In [10]:
for name, group in df.groupby("key1"):
    print(name)
    print("*"*60)
    print(group)

a
************************************************************
  key1 key2  value1  value2
0    a  one       0       2
1    a  two       1       6
4    a  one       4      12
b
************************************************************
  key1 key2  value1  value2
2    b  one       2       8
3    b  two       3      10


- **多重键情况下，分组迭代**。此时，元组的第一个元素将会是由键值组成的元组

In [11]:
for name, group in df.groupby(["key1", "key2"]):
    print(name)
    print("*"*60)
    print(group)

('a', 'one')
************************************************************
  key1 key2  value1  value2
0    a  one       0       2
4    a  one       4      12
('a', 'two')
************************************************************
  key1 key2  value1  value2
1    a  two       1       6
('b', 'one')
************************************************************
  key1 key2  value1  value2
2    b  one       2       8
('b', 'two')
************************************************************
  key1 key2  value1  value2
3    b  two       3      10


- **可以将分组的数据片段做成字典进行访问**

In [12]:
pieces = dict(list(df.groupby("key1")))
pieces

{'a':   key1 key2  value1  value2
 0    a  one       0       2
 1    a  two       1       6
 4    a  one       4      12, 'b':   key1 key2  value1  value2
 2    b  one       2       8
 3    b  two       3      10}

In [13]:
pieces["a"]

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
4,a,one,4,12


 - **groupby默认在axis=0上分组，通过设置也可以在其他任何轴上进行分组。**

In [14]:
# 根据dtype对列进行分组
for dtype, group in df.groupby(df.dtypes, axis=1):
    print(dtype)
    print("*"*60)
    print(group)

int64
************************************************************
   value1  value2
0       0       2
1       1       6
2       2       8
3       3      10
4       4      12
object
************************************************************
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


##### 选取一列或列的子集

对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。

In [7]:
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


In [8]:
df.groupby(["key1", "key2"])[["value2"]].mean()

value2
key1 key2        
a    one        7
     two        6
b    one        8
     two       10

In [14]:
type(df.groupby(["key1", "key2"])[["value2"]].mean())

pandas.core.frame.DataFrame

##### 通过字典或Series进行分组

In [19]:
mapping = {
    "key1" : "orange",
    "key2" : "blue",
    "value1" : "red",
    "value2" : "red"
}
df.groupby(mapping, axis=1).sum()

,blue,orange,red
0,one,a,2
1,two,a,7
2,one,b,10
3,two,b,13
4,one,a,16


In [20]:
df.groupby(pd.Series(mapping), axis=1).count()

,blue,orange,red
0,1,1,2
1,1,1,2
2,1,1,2
3,1,1,2
4,1,1,2


##### 通过函数进行分组

任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称。

In [29]:
df = pd.DataFrame({
    "key1" : ['a', 'a', 'b', 'b', 'a'],
    "key2" : ['one', 'two', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4],
    "value2" : [2, 6, 8, 10, 12]
}, index = ["First", "Second", "Third", "Fourth", "Fifth"])
df

,key1,key2,value1,value2
First,a,one,0,2
Second,a,two,1,6
Third,b,one,2,8
Fourth,b,two,3,10
Fifth,a,one,4,12


In [30]:
df.groupby(len).sum()

,value1,value2
5,6,22
6,4,16


In [32]:
key_list = ['one', 'one', 'one', 'two', 'two']
df.groupby([len, key_list]).sum()

value1  value2
5 one       2      10
  two       4      12
6 one       1       6
  two       3      10

##### 根据索引级别分组

层次化索引数据集最方便的地方是它可以根据索引的一个级别进行聚合。

In [34]:
columns = pd.MultiIndex.from_arrays([ \
    ["US", "US", "CH", "CH"],[1, 5, 1, 3]], \
    names=["city", "tenor"])

df = pd.DataFrame([
    ["a", "one", 0, 2],
    ["a", "two", 1, 6],
    ["b", "two", 2, 8],
    ["b", "two", 3, 10],
    ["a", "one", 4, 12]
], columns = columns)
df

city  US      CH    
tenor  1    5  1   3
0      a  one  0   2
1      a  two  1   6
2      b  two  2   8
3      b  two  3  10
4      a  one  4  12

- 根据level关键字传递级别序号或名字，实现根据界别分组

In [38]:
df.groupby(level="city", axis = 1).sum()

city,CH,US
0,2,aone
1,7,atwo
2,10,btwo
3,13,btwo
4,16,aone


#### 数据聚合

聚合指的是任何能够从数组产生标量值的数据转换过程，比如
- count:分组中非NAN值的数量
- sum:非NAN值的平均值
- mean:非NAN值的平均数
- median:非NAN值的算数中位数
- std、var:无偏(分母为n-1)标准差和方差
- min、max:非NAN值的最大值和最小值
- prod:非NAN值的积
- first、last:第一个和最后一个非NAN的值

In [21]:
df = pd.DataFrame({
    "key1" : ['a', 'a', 'b', 'b', 'a'],
    "key2" : ['one', 'two', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4],
    "value2" : [2, 6, 8, 10, 12]
})
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


- **quantile可以计算Series或DataFrame列的样本分位数。**

In [22]:
df.groupby("key1")["value1"].quantile(0.9)

key1
a    3.4
b    2.9
Name: value1, dtype: float64

In [23]:
df.groupby("key1")["value1"].median()

key1
a    1.0
b    2.5
Name: value1, dtype: float64

- **将自定义聚合函数传入aggregate或agg方法实现自定义聚合操作**

自定义聚合函数要比sum等经过优化的函数慢得多。这是因为在构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）。

In [24]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
df.groupby("key1")["value1"].agg(peak_to_peak)

key1
a    4
b    1
Name: value1, dtype: int64

- **某些方法也可以用于GroupBy变量**

In [25]:
df.groupby("key1")["value1"].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,1.666667,2.081666,0.0,0.50,1.0,2.50,4.0
b,2.0,2.500000,0.707107,2.0,2.25,2.5,2.75,3.0


#### 面向列的多函数应用

In [26]:
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


In [27]:
df["percent"] = df["value1"] / df["value2"]
df

,key1,key2,value1,value2,percent
0,a,one,0,2,0.000000
1,a,two,1,6,0.166667
2,b,one,2,8,0.250000
3,b,two,3,10,0.300000
4,a,one,4,12,0.333333


- **对不同的列使用不同的聚合函数**

In [28]:
df.groupby(["key1", "key2"]).agg(["mean", "sum"])

value1     value2       percent          
            mean sum   mean sum      mean       sum
key1 key2                                          
a    one       2   4      7  14  0.166667  0.333333
     two       1   1      6   6  0.166667  0.166667
b    one       2   2      8   8  0.250000  0.250000
     two       3   3     10  10  0.300000  0.300000

In [29]:
df.groupby(["key1", "key2"]).agg([("key1","mean"), ("key2", "sum")])

value1      value2        percent          
            key1 key2   key1 key2      key1      key2
key1 key2                                            
a    one       2    4      7   14  0.166667  0.333333
     two       1    1      6    6  0.166667  0.166667
b    one       2    2      8    8  0.250000  0.250000
     two       3    3     10   10  0.300000  0.300000

In [32]:
df.groupby(["key1", "key2"]).agg(["mean", "sum"])

value1     value2       percent          
            mean sum   mean sum      mean       sum
key1 key2                                          
a    one       2   4      7  14  0.166667  0.333333
     two       1   1      6   6  0.166667  0.166667
b    one       2   2      8   8  0.250000  0.250000
     two       3   3     10  10  0.300000  0.300000

In [33]:
df.groupby(["key1", "key2"])["value1"].agg(["mean", "sum"])

mean  sum
key1 key2           
a    one      2    4
     two      1    1
b    one      2    2
     two      3    3

- **对一或多列应用不同函数，可以通过传入一个从列名映射到函数的字典。**

In [34]:
df.groupby(["key1", "key2"]).agg({"value1":np.sum, "value2" : "mean"})

value1  value2
key1 key2                
a    one        4       7
     two        1       6
b    one        2       8
     two        3      10

#### **“没有行索引”的形式返回聚合数据**


向groupby传入as_index=False以禁用聚合数据都有由唯一的分组键组成的索引.
对结果调用reset_index也能得到这种形式的结果。使用as_index=False方法可以避免一些不必要的计算。

In [35]:
df

,key1,key2,value1,value2,percent
0,a,one,0,2,0.000000
1,a,two,1,6,0.166667
2,b,one,2,8,0.250000
3,b,two,3,10,0.300000
4,a,one,4,12,0.333333


In [43]:
df.groupby(["key1", "key2"], as_index=False, axis=0).sum()

,key1,key2,value1,value2,percent
0,a,one,4,14,0.333333
1,a,two,1,6,0.166667
2,b,one,2,8,0.250000
3,b,two,3,10,0.300000


### apply:一般性的“拆分－应用－合并”

最通用的GroupBy方法是apply，apply会将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试将各片段组合到一起。

In [51]:
df = pd.DataFrame({
    "key1" : ['a', 'c', 'b', 'b', 'a', "d", "b", "a"],
    "key2" : ['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4, 8, 5, 6],
    "value2" : [2, 6, 8, 10, 12, 16, 9, 34]
})
df

,key1,key2,value1,value2
0,a,one,0,2
1,c,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12
5,d,one,8,16
6,b,two,5,9
7,a,one,6,34


In [52]:
def topN(df, n=3,column="value1"):
    return df.sort_values(by=column)[-n:]
topN(df)

,key1,key2,value1,value2
6,b,two,5,9
7,a,one,6,34
5,d,one,8,16


- **对分组并进行函数调用。**

In [53]:
df.groupby("key1").apply(topN)

key1 key2  value1  value2
key1                            
a    0    a  one       0       2
     4    a  one       4      12
     7    a  one       6      34
b    2    b  one       2       8
     3    b  two       3      10
     6    b  two       5       9
c    1    c  two       1       6
d    5    d  one       8      16

topN函数在DataFrame的各个片段上调用，然后结果由pandas.concat组装到一起，并以分组名称进行了标记，最终结果就形成一个层次化索引，其内层索引值来自原DataFrame。

- **apply函数接受其他参数输入。**

In [54]:
df.groupby("key1").apply(topN, n=2, column="value2")

key1 key2  value1  value2
key1                            
a    4    a  one       4      12
     7    a  one       6      34
b    6    b  two       5       9
     3    b  two       3      10
c    1    c  two       1       6
d    5    d  one       8      16

- **groupBy对象上调用过describe**

In [59]:
df.groupby("key1").apply(topN, n=2, column="value2").describe()

,value1,value2
count,6.000000,6.000000
mean,4.500000,14.500000
std,2.428992,10.114346
min,1.000000,6.000000
25%,3.250000,9.250000
50%,4.500000,11.000000
75%,5.750000,15.000000
max,8.000000,34.000000


In [64]:
df.groupby("key1").apply(topN, n=2, column="value2")[["value1", "value2"]].apply(lambda x: x.describe())

,value1,value2
count,6.000000,6.000000
mean,4.500000,14.500000
std,2.428992,10.114346
min,1.000000,6.000000
25%,3.250000,9.250000
50%,4.500000,11.000000
75%,5.750000,15.000000
max,8.000000,34.000000


#### 禁止分组键

分组键会跟原始对象的索引共同构成结果对象中的层次化索引。将group_keys=False传入groupby即可禁止该效果.

In [67]:
df.groupby("key1", group_keys = True).apply(topN)

key1 key2  value1  value2
key1                            
a    0    a  one       0       2
     4    a  one       4      12
     7    a  one       6      34
b    2    b  one       2       8
     3    b  two       3      10
     6    b  two       5       9
c    1    c  two       1       6
d    5    d  one       8      16

In [66]:
df.groupby("key1", group_keys = False).apply(topN)

,key1,key2,value1,value2
0,a,one,0,2
4,a,one,4,12
7,a,one,6,34
2,b,one,2,8
3,b,two,3,10
6,b,two,5,9
1,c,two,1,6
5,d,one,8,16


#### 分位数和桶分析

pandas有一些能根据指定面元或样本分位数将数据拆分成多块的工具（比如cut和qcut）。将这些函数跟groupby结合起来，就能非常轻松地实现对数据集的桶（bucket）或分位数（quantile）分析。

In [68]:
df = pd.DataFrame({
    "key1" : ['a', 'c', 'b', 'b', 'a', "d", "b", "a"],
    "key2" : ['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4, 8, 5, 6],
    "value2" : [2, 6, 8, 10, 12, 16, 9, 34]
})
df

,key1,key2,value1,value2
0,a,one,0,2
1,c,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12
5,d,one,8,16
6,b,two,5,9
7,a,one,6,34


- **由cut返回的Categorical对象可直接传递到groupby**

In [80]:
quartiles = pd.cut(df["value1"], 4)
quartiles

0    (-0.008, 2.0]
1    (-0.008, 2.0]
2    (-0.008, 2.0]
3       (2.0, 4.0]
4       (2.0, 4.0]
5       (6.0, 8.0]
6       (4.0, 6.0]
7       (4.0, 6.0]
Name: value1, dtype: category
Categories (4, interval[float64]): [(-0.008, 2.0] < (2.0, 4.0] < (4.0, 6.0] < (6.0, 8.0]]

In [81]:
df.groupby(quartiles).apply(np.min).unstack()

        value1       
key1    (-0.008, 2.0]      a
        (2.0, 4.0]         a
        (4.0, 6.0]         a
        (6.0, 8.0]         d
key2    (-0.008, 2.0]    one
        (2.0, 4.0]       one
        (4.0, 6.0]       one
        (6.0, 8.0]       one
value1  (-0.008, 2.0]      0
        (2.0, 4.0]         3
        (4.0, 6.0]         5
        (6.0, 8.0]         8
value2  (-0.008, 2.0]      2
        (2.0, 4.0]        10
        (4.0, 6.0]         9
        (6.0, 8.0]        16
dtype: object

In [88]:
def get_stats(group):
    return {
        "min" : group.min(),
        "max" : group.max(),
        "count" : group.count(),
        "mean" : group.mean()
    }
df["value2"].groupby(quartiles).apply(get_stats).unstack()

,count,max,mean,min
value1,,,,
"(-0.008, 2.0]",3.0,8.0,5.333333,2.0
"(2.0, 4.0]",2.0,12.0,11.000000,10.0
"(4.0, 6.0]",2.0,34.0,21.500000,9.0
"(6.0, 8.0]",1.0,16.0,16.000000,16.0


#### 透视表和交叉表

##### 透视表

In [106]:
df = pd.DataFrame({
    "key1" : ['a', 'c', 'b', 'b', 'a', "d", "b", "a"],
    "key2" : ['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4, 8, 5, 6],
    "value2" : [2, 6, 8, 10, 12, 16, 9, 34]
})
df

,key1,key2,value1,value2
0,a,one,0,2
1,c,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12
5,d,one,8,16
6,b,two,5,9
7,a,one,6,34


In [90]:
df.pivot_table(index=["key1", "key2"])

value1  value2
key1 key2                  
a    one   3.333333    16.0
b    one   2.000000     8.0
     two   4.000000     9.5
c    two   1.000000     6.0
d    one   8.000000    16.0

In [95]:
df.pivot_table(["value1"], index=["key1", "key2"])

value1
key1 key2          
a    one   3.333333
b    one   2.000000
     two   4.000000
c    two   1.000000
d    one   8.000000

- **传入margins=True，添加分项小计**,之后将会添加标签为All的行和列，其值对应于单个等级中所有数据的分组统计.

In [97]:
df.pivot_table(["value1","value2"], index=["key1", "key2"], margins=True)

value1  value2
key1 key2                  
a    one   3.333333  16.000
b    one   2.000000   8.000
     two   4.000000   9.500
c    two   1.000000   6.000
d    one   8.000000  16.000
All        3.625000  12.125

- **通过aggfunc传入聚合函数**

In [101]:
df.pivot_table(["value1","value2"],index=["key1", "key2"], 
        margins=True, aggfunc=len)

value1  value2
key1 key2                
a    one        3       3
b    one        1       1
     two        2       2
c    two        1       1
d    one        1       1
All             8       8

In [107]:
df["value1"][2:4] = np.nan
df

D:\soft\Programs\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,key1,key2,value1,value2
0,a,one,0.0,2
1,c,two,1.0,6
2,b,one,NaN,8
3,b,two,NaN,10
4,a,one,4.0,12
5,d,one,8.0,16
6,b,two,5.0,9
7,a,one,6.0,34


- **如果存在Na，可以通过fill_value进行值的设置。**

In [109]:
df.pivot_table(["value1","value2"],index=["key1", "key2"], fill_value =0)

value1  value2
key1 key2                  
a    one   3.333333    16.0
b    one   0.000000     8.0
     two   5.000000     9.5
c    two   1.000000     6.0
d    one   8.000000    16.0

##### 交叉表:crosstab

用于计算分组频率的特殊透视表

In [110]:
df = pd.DataFrame({
    "key1" : ['a', 'c', 'b', 'b', 'a', "d", "b", "a"],
    "key2" : ['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4, 8, 5, 6],
    "value2" : [2, 6, 8, 10, 12, 16, 9, 34]
})
df

,key1,key2,value1,value2
0,a,one,0,2
1,c,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12
5,d,one,8,16
6,b,two,5,9
7,a,one,6,34


In [115]:
pd.crosstab(df.key1, df.value1, margins=True)

value1,0,1,2,3,4,5,6,8,All
key1,,,,,,,,,
a,1,0,0,0,1,0,1,0,3
b,0,0,1,1,0,1,0,0,3
c,0,1,0,0,0,0,0,0,1
d,0,0,0,0,0,0,0,1,1
All,1,1,1,1,1,1,1,1,8


- **crosstab的前两个参数可以是数组或Series，或数组列**

In [116]:
pd.crosstab([df.key1, df.key2], df.value1, margins=True)

value1     0  1  2  3  4  5  6  8  All
key1 key2                             
a    one   1  0  0  0  1  0  1  0    3
b    one   0  0  1  0  0  0  0  0    1
     two   0  0  0  1  0  1  0  0    2
c    two   0  1  0  0  0  0  0  0    1
d    one   0  0  0  0  0  0  0  1    1
All        1  1  1  1  1  1  1  1    8